In [1]:
import importlib
import gym_model
importlib.reload(gym_model)
from visualisation import JupyterVizGym, GymDrawer, draw_layout
from optimization import LayoutTemplate
import random
import numpy as np          
import matplotlib.pyplot as plt

In [10]:
model_params = {
    "interarrival_time": {
        "type": "SliderInt",
        "value": 5,
        "label": "time between arrivals:",
        "min": 1,
        "max": 50,
        "step": 1,
    },
    "agent_exercise_duration": {
        "type": "SliderInt",
        "value": 30,
        "label": "mean exercise duration:",
        "min": 1,
        "max": 3 * 60,
        "step": 1,
    },
}

RANDOM_LAYOUT = False

if RANDOM_LAYOUT:
    model_params["machine_density"] ={
        "type": "SliderFloat",
        "value": 0.25,
        "label": "Density of gym equipment:",
        "min": 0.05,
        "max": 1,
        "step": 0.05,
    }
else:
    template = LayoutTemplate.square_isles(isle_rows=2, isle_cols=3)
    model_params["layout"] = template.instantiate([random.choice(list(gym_model.Equipment)) for _ in range(len(template))])
    model_params["spawn_location"] = template.entrance


page = JupyterVizGym(
    gym_model.Gym,
    model_params,
    measures=["Utilization"],
    name="Test gym",
    agent_portrayal=lambda agent: agent.portrayal,
    # space_drawer="default",
    space_drawer=GymDrawer,
)

page

Cannot show ipywidgets in text

In [11]:
eff = np.load("layouts\\best_1x2_gym_efficiency.npy", allow_pickle=True)
fig, ax = plt.subplots(figsize=(10,10))
draw_layout(eff, ax)
plt.show()

In [2]:
from optimization import optimal_gym, GymMetric, LayoutTemplate

template = LayoutTemplate.square_isles(isle_rows=2, isle_cols=2)
layout, fitness, ga_instance = optimal_gym(
    layout_template=template,
    fitness_metrics=[GymMetric.UTILIZATION, GymMetric.EFFICIENCY, GymMetric.CONGESTION],
    n_generations=20, population_size=10,
    crossover_method="two_points", tournament_participants=None,
    mutation_percents=10,
    plot_fitness=False,
    simulation_cycle_steps=25, interarrival_time=1, agent_exercise_duration=5,
)

----------------------------------------------------------------------
                           PyGAD Lifecycle                           
Step                  Handler                              Output Shape
Fitness Function      gym_quality()                        (1)      
----------------------------------------------------------------------
Parent Selection      nsga2_selection()                    (5, 47)  
Number of Parents: 5
----------------------------------------------------------------------
Crossover             two_points_crossover()               (5, 47)  
----------------------------------------------------------------------
Mutation              random_mutation()                    (5, 47)  
Mutation Genes: 4
Random Mutation Range: (0, 17)
Gene Space: range(0, 18)
Mutation by Replacement: True
Allow Duplicated Genes: True
----------------------------------------------------------------------
On Generation         print_evolution_progress()           None     
----

INFO: discarded 12/10 solutions
Finished generation 1/20, (lexi)max fitness=[0.1506383  0.47201839 0.51844733]
INFO: discarded 3/10 solutions
Finished generation 2/20, (lexi)max fitness=[0.12680851 0.56233014 0.40366234]
INFO: discarded 4/10 solutions
Finished generation 3/20, (lexi)max fitness=[0.10297872 0.5900828  0.39651082]
INFO: discarded 3/10 solutions
Finished generation 4/20, (lexi)max fitness=[0.10297872 0.5900828  0.39651082]
INFO: discarded 1/10 solutions
Finished generation 5/20, (lexi)max fitness=[0.10297872 0.5900828  0.39651082]
INFO: discarded 2/10 solutions
Finished generation 6/20, (lexi)max fitness=[0.12765957 0.60912955 0.44822511]
INFO: discarded 2/10 solutions
Finished generation 7/20, (lexi)max fitness=[0.12765957 0.60912955 0.44822511]
INFO: discarded 2/10 solutions
Finished generation 8/20, (lexi)max fitness=[0.13702128 0.63387444 0.4355873 ]
INFO: discarded 4/10 solutions
Finished generation 9/20, (lexi)max fitness=[0.13702128 0.63387444 0.4355873 ]
INFO: dis

In [3]:
fit = ga_instance.last_generation_fitness
print(fit)
np.where(fit == np.max(fit))

[[0.1506383  0.47201839 0.51844733]
 [0.12       0.6432903  0.46339683]
 [0.10468085 0.47112276 0.63611411]
 [0.12340426 0.6418192  0.37301587]
 [0.14638298 0.60415306 0.40148341]
 [      -inf       -inf       -inf]
 [0.0987234  0.63449692 0.42537662]
 [0.10382979 0.55911493 0.49266378]
 [      -inf       -inf       -inf]
 [0.13787234 0.61221092 0.41832035]]


(array([1], dtype=int64), array([1], dtype=int64))

In [4]:
ga_instance.best_solution()

(array([12, 17, 16, 15, 15,  1, 16, 17,  8,  4,  2,  6, 15,  5,  0, 14,  6,
         8, 13, 12, 10, 11, 15, 13,  3, 16,  9,  5, 12, 16, 12,  7,  8,  8,
        13,  0,  4,  3,  1,  4,  5, 16,  7,  6,  6, 12,  9]),
 array([0.12      , 0.6432903 , 0.46339683]),
 1)

In [11]:
gym = gym_model.Gym(interarrival_time=10, agent_exercise_duration=30)

model_df, agent_df = gym.run(1000, progress_bar=True)
agent_df

Simulating gym: 100%|██████████| 1000/1000 [00:00<00:00, 1443.09it/s, 15 agents]


State
Step AgentID             
0    0          SEARCHING
1    0        WORKING_OUT
2    0        WORKING_OUT
3    0        WORKING_OUT
4    0        WORKING_OUT
...                   ...
999  96       WORKING_OUT
     97       WORKING_OUT
     98       WORKING_OUT
     99       WORKING_OUT
     100        SEARCHING

[13335 rows x 1 columns]